Various GWAS genesets are tested for enrichment in SCZ somatic variants.

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
from bsmcalls import SNPnexus
from bsmcalls import operations
from bsmcalls import resources
from bsmcalls import preprocessing
from matplotlib import pyplot as plt
import re
import attila_utils
%matplotlib inline

In [2]:
data = SNPnexus.load_data('../../results/2020-09-07-annotations/annotated-calls.p')

## Autism spectrum disorder (ASD)
Read autism GWAS loci from the GWAS Catalog to a DataFrame

In [3]:
gwaspath = '../../resources/GWAS/autism-spectrum-disorder.tsv'
asdloci = pd.read_csv(gwaspath, sep='\t')

We will select the study ``Identification of common genetic risk variants for autism spectrum disorder.'' (PMID [30804558](www.ncbi.nlm.nih.gov/pubmed/30804558)).

In [4]:
asdloci = asdloci.loc[asdloci['PUBMEDID'] == 30804558]
selcols = ['CHR_ID', 'CHR_POS', 'REPORTED GENE(S)', 'MAPPED_GENE']
asdloci[selcols]

,CHR_ID,CHR_POS,REPORTED GENE(S),MAPPED_GENE
94,1,96096246,"LOC102723661, PTBP2",LINC02790 - RNU1-130P
95,1,96513405,Intergenic,EEF1A1P11 - RN7SL831P
96,1,98627228,Intergenic,LINC01776 - AC095031.1
97,1,104249635,Intergenic,AC092506.1 - AL591888.1
98,1,193533479,Intergenic,AL136456.1
...,...,...,...,...
151,20,21267478,"KIZ, XRN2, NKX2-2, NKX2-4",ZNF877P - AL117332.1
152,5,104676602,NaN,AC099520.1
153,14,103551616,NaN,RNU7-160P - AL139300.2
154,1,72263459,NaN,NEGR1


In [5]:
reported = set(asdloci['REPORTED GENE(S)'].dropna().str.split(', ').sum())
reported.remove('Intergenic')
reported

{'C8orf74',
 'CADPS',
 'KIZ',
 'KMT2E',
 'LOC102723661',
 'MACROD2',
 'MROH5',
 'NKX2-2',
 'NKX2-4',
 'PINX1',
 'PTBP2',
 'SOX7',
 'SRPK2',
 'XRN2'}

In [6]:
mappeds = asdloci['MAPPED_GENE'].dropna()
commas = mappeds.str.match('^.*, .*$')
mappeds1 = mappeds.loc[commas]
mapped1 = set(mappeds1.str.split(', ').sum())
mappeds2 = mappeds.loc[~ commas]
mapped2 = set(mappeds2.str.split(' - ').sum())
mapped = mapped2.union(mapped1)
asd_gwas_genes = mapped.union(reported)
print(len(asd_gwas_genes), 'genes')
print(asd_gwas_genes)

79 genes
{'NRGN', 'AL022724.3', 'NKX2-4', 'AL117332.1', 'RNU1-130P', 'MAPT', 'FGF12', 'AC092506.1', 'PINX1', 'LINC01776', 'AC022387.1', 'ZNF877P', 'AC120036.1', 'AL589740.1', 'LINC02790', 'AL032822.1', 'ITIH3', 'GUCY1A2', 'LINC00461', 'AL391117.1', 'MROH5', 'U95743.1', 'AC090987.1', 'EGR2', 'NEGR1', 'SERPINA1', 'ZNF568', 'AC099520.1', 'SOX7', 'AL035078.4', 'AL133270.1', 'RBFOX1', 'PAUPAR', 'PTBP2', 'KCNN2', 'AL136456.1', 'CADPS', 'AC025839.1', 'BEND4', 'NEDD4L', 'KIZ', 'OLFM4', 'PKP4', 'NKX2-2', 'GABBR1', 'RSRC1', 'SRPK2', 'AL591888.1', 'LINC02058', 'PCDH9', 'RN7SL831P', 'MACROD2', 'AC005070.3', 'AC095031.1', 'U91319.1', 'SGO1-AS1', 'EEF1A1P11', 'C8orf74', 'RSU1', 'AL139093.1', 'AC026320.3', 'ADTRP', 'ITIH4', 'SORCS3', 'KANSL1', 'ESRRB', 'ASAP1', 'KMT2E', 'GALNT10', 'AC013287.1', 'RP1L1', 'LOC102723661', 'LINC02210-CRHR1', 'AL139300.2', 'AC021192.2', 'XRN2', 'RNU7-160P', 'SERPINA2', 'GALNT1'}


Let us read in the set of SCZ GWAS genes!

In [7]:
clozukpath = '../../resources/CLOZUK/supp-table-4.csv'
scz_gwas_genes = operations.get_geneset(df=pd.read_csv(clozukpath, skiprows=7), col='Gene(s) tagged')

In [8]:
querydict = {'near_gens_Annotation': ['coding nonsyn', 'coding syn', 'missense', 'stop-gain', 'intronic (splice_site)'],
             'ensembl_Predicted Function': ['coding'],
             'sift_Prediction': ['Deleterious', 'Deleterious - Low Confidence'],
             'polyphen_Prediction': ['Probably Damaging', 'Possibly Damaging'],
             'tfbs_TFBS Name': None,
             'phast_Score': None,
             'gerp_Element RS Score': None,
             'cpg_CpG Island': None,
             'near_gens_Overlapped Gene': {'SCZ GWAS genes': scz_gwas_genes,
                                           'ASD GWAS genes': asd_gwas_genes,
                                           'ASD reported GWAS genes': reported,
                                          },
            }
results = operations.multiquery(querydict, data, do_sum=False, do_sort=False)
summary = operations.summarize_query_results(results, data, chisq=True, margin=False)
summary#.style.bar(subset='chisq stat')

/home/attila/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/home/attila/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:6125: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs.astype(np.float64) - f_exp)**2 / f_exp


Control  SCZ  ASD  \
Feature                    Query                                             
near_gens_Annotation       coding nonsyn                      23   43   23   
                           coding syn                         16   20   15   
                           missense                            0    0    0   
                           stop-gain                           1    0    0   
                           intronic (splice_site)              0    1    0   
ensembl_Predicted Function coding                             39   64   40   
sift_Prediction            Deleterious                         8    9    6   
                           Deleterious - Low Confidence        1    1    2   
polyphen_Prediction        Probably Damaging                   3    5    2   
                           Possibly Damaging                   2    2    0   
tfbs_TFBS Name             any                                48   78   50   
phast_Score                any                               102  174  141   
gerp_Element RS Score      any                               140  262  208   
cpg_CpG Island             any                                26   47   19   
near_gens_Overlapped Gene  SCZ GWAS genes                     20   66   42   
                           ASD GWAS genes                      2   20    7   
                           ASD reported GWAS genes             0    6    0   

                                                         chisq stat   chisq p  
Feature                    Query                                               
near_gens_Annotation       coding nonsyn                   4.480340  0.106440  
                           coding syn                      1.177849  0.554924  
                           missense                             NaN       NaN  
                           stop-gain                       2.887477  0.236044  
                           intronic (splice_site)          1.582797  0.453210  
ensembl_Predicted Function coding                          3.794387  0.149989  
sift_Prediction            Deleterious                     1.315056  0.518131  
                           Deleterious - Low Confidence    0.429823  0.806613  
polyphen_Prediction        Probably Damaging               1.080625  0.582566  
                           Possibly Damaging               2.470275  0.290795  
tfbs_TFBS Name             any                             4.120011  0.127453  
phast_Score                any                             1.591145  0.451323  
gerp_Element RS Score      any                             5.012216  0.081585  
cpg_CpG Island             any                             9.614759  0.008169  
near_gens_Overlapped Gene  SCZ GWAS genes                 10.800565  0.004515  
                           ASD GWAS genes                 11.912735  0.002589  
                           ASD reported GWAS genes         9.496785  0.008666

## Psychiatric disorders

The [Analysis of shared heritability in common disorders of the brain](https://pubmed.ncbi.nlm.nih.gov/29930110/) showed the following psychiatric disorders to correlate significantly with schizophrenia or ASD:
* ADHD; PMID: [30478444](https://pubmed.ncbi.nlm.nih.gov/30478444/); Demontis et al 2019 Nat Genet
* Anorexia nervosa; PMID: [31308545](https://pubmed.ncbi.nlm.nih.gov/31308545/); Watson et al 2019 Nat Genet
* ASD; PMID: [30804558](https://pubmed.ncbi.nlm.nih.gov/30804558/); Grove et al 2019 Nat Genet
* Bipolar disorder; PMID: [31043756](https://pubmed.ncbi.nlm.nih.gov/31043756/); Stahl et al 2019 Nat Genet
* MDD; PMID: [30718901](https://pubmed.ncbi.nlm.nih.gov/30718901/); Howard et al 2019 Nat Neurosci
* OCD;  PMID: [31891238](https://pubmed.ncbi.nlm.nih.gov/31891238/); Smit et al 2020 Am J Med Genet B Neuropsychiatr Genet
* Schizophrenia; PMID: [29483656](https://pubmed.ncbi.nlm.nih.gov/29483656/); Pardiñas et al 2018 Nat Genet

The following psychiatric disorders do not correlate with schizophrenia nor ASD:
* Anxdiety disorders
* OCD
* PTSD
* Tourette syndrome; PMID: [30818990](https://pubmed.ncbi.nlm.nih.gov/30818990/); Yu et al 2019 Am J Psychiatry

None of the following neurological diseases showed significant correlation with schizophrenia or ASD:
* Alzheimer's disease
* Epilepsy
* Focal epilepsy
* Generalized epilepsy
* ICH
* Ischemic stroke
* Early-onset stroke
* Migraine
* Migraine with aura
* Migraine without aura
* Multiple schlerosis
* Parkinson's disease

In [9]:
fpath = '../2021-03-09-gwas-genes/selected-gwas.csv'
selgwas = pd.read_csv(fpath, index_col='Trait', sep='\t')
selgwas

,Trait type,PMID,Citation
Trait,,,
ADHD,psychiatric,30478444,Demontis et al 2019 Nat Genet
Anorexia nervosa,psychiatric,31308545,Watson et al 2019 Nat Genet
Anxiety disorders,psychiatric uncorr,31116379,Meier et al 2019 JAMA Psychiatry
ASD,psychiatric,30804558,Meier et al 2019 JAMA Psychiatry
Bipolar disorder,psychiatric,31043756,Stahl et al 2019 Nat Genet
MDD,psychiatric,30718901,Howard et al 2019 Nat Neurosci
OCD,psychiatric uncorr,30456828,Khramtsova et al 2018 Am J Med Genet
PTSD,psychiatric uncorr,31594949,Nievergelt et al 2019 Nat Commun
Schizophrenia,psychiatric,29483656,Pardiñas et al 2018 Nat Genet


In [10]:
fpath = '../../resources/GWAS/gwas_catalog_v1.0.2-associations_e100_r2021-02-25.tsv'
gwas = pd.read_csv(fpath, sep='\t')
gwas = gwas.loc[gwas['PUBMEDID'].isin(selgwas['PMID']), :]

/home/attila/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (9,11,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
psychiatric_genesets = resources.gwascat_multi_genesets(gwas, selgwas.loc[selgwas['Trait type'] == 'psychiatric'])

psychiatric_reported = set()
psychiatric = set()
for k in psychiatric_genesets.keys():
    gset = psychiatric_genesets[k]
    if re.match('.*reported$', k):
        psychiatric_reported.update(gset)
    else:
        psychiatric.update(gset)
    bname = k.replace(' ', '-') + '.tsv'
    fpath = '../../results/2021-03-09-gwas-genes/' + bname
    pd.DataFrame({k: sorted(list(psychiatric_genesets[k]))}).to_csv(fpath, sep='\t', index=False)

print('Number of genes in associated to any psychiatric disorder:')
print(len(psychiatric_reported), 'reported')
print(len(psychiatric), 'reported or mapped')

psychiatric_genesets.update({'Psychiatric reported': psychiatric_reported, 'Psychiatric': psychiatric})

Number of genes in associated to any psychiatric disorder:
894 reported
1270 reported or mapped


In [12]:
querydict = {'near_gens_Annotation': ['coding nonsyn', 'coding syn', 'missense', 'stop-gain', 'intronic (splice_site)'],
             'ensembl_Predicted Function': ['coding'],
             'sift_Prediction': ['Deleterious', 'Deleterious - Low Confidence'],
             'polyphen_Prediction': ['Probably Damaging', 'Possibly Damaging'],
             'tfbs_TFBS Name': None,
             'phast_Score': None,
             'gerp_Element RS Score': None,
             'cpg_CpG Island': None,
             'near_gens_Overlapped Gene': psychiatric_genesets,
            }
results = operations.multiquery(querydict, data, do_sum=False, do_sort=False)
summary = operations.summarize_query_results(results, data, chisq=True, margin=False)
summary#.style.bar(subset='chisq stat')

/home/attila/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/home/attila/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:6125: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs.astype(np.float64) - f_exp)**2 / f_exp


Control  SCZ  ASD  \
Feature                    Query                                             
near_gens_Annotation       coding nonsyn                      23   43   23   
                           coding syn                         16   20   15   
                           missense                            0    0    0   
                           stop-gain                           1    0    0   
                           intronic (splice_site)              0    1    0   
ensembl_Predicted Function coding                             39   64   40   
sift_Prediction            Deleterious                         8    9    6   
                           Deleterious - Low Confidence        1    1    2   
polyphen_Prediction        Probably Damaging                   3    5    2   
                           Possibly Damaging                   2    2    0   
tfbs_TFBS Name             any                                48   78   50   
phast_Score                any                               102  174  141   
gerp_Element RS Score      any                               140  262  208   
cpg_CpG Island             any                                26   47   19   
near_gens_Overlapped Gene  ADHD reported                       2    3    1   
                           ADHD                                2    3    1   
                           Anorexia nervosa reported           1    1    2   
                           Anorexia nervosa                    1    1    2   
                           ASD reported                        0    6    0   
                           ASD                                 2   20    7   
                           Bipolar disorder reported          23   53   26   
                           Bipolar disorder                   28   63   30   
                           MDD reported                        0    0    0   
                           MDD                                13   27   22   
                           Schizophrenia reported             11   43   33   
                           Schizophrenia                      11   43   36   
                           Psychiatric reported               34   97   58   
                           Psychiatric                        49  127   80   

                                                         chisq stat   chisq p  
Feature                    Query                                               
near_gens_Annotation       coding nonsyn                   4.480340  0.106440  
                           coding syn                      1.177849  0.554924  
                           missense                             NaN       NaN  
                           stop-gain                       2.887477  0.236044  
                           intronic (splice_site)          1.582797  0.453210  
ensembl_Predicted Function coding                          3.794387  0.149989  
sift_Prediction            Deleterious                     1.315056  0.518131  
                           Deleterious - Low Confidence    0.429823  0.806613  
polyphen_Prediction        Probably Damaging               1.080625  0.582566  
                           Possibly Damaging               2.470275  0.290795  
tfbs_TFBS Name             any                             4.120011  0.127453  
phast_Score                any                             1.591145  0.451323  
gerp_Element RS Score      any                             5.012216  0.081585  
cpg_CpG Island             any                             9.614759  0.008169  
near_gens_Overlapped Gene  ADHD reported                   0.934557  0.626706  
                           ADHD                            0.934557  0.626706  
                           Anorexia nervosa reported       0.429823  0.806613  
                           Anorexia nervosa                0.429823  0.806613  
                           ASD reported                    9.496785  0.008666  
                           ASD      

The same without ASD

In [13]:
results = operations.multiquery(querydict, data, do_sum=False, do_sort=False)
summary = operations.summarize_query_results(results, preprocessing.drop_category(data, 'ASD'), chisq=True, margin=False)
summary

/home/attila/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/home/attila/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:6125: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs.astype(np.float64) - f_exp)**2 / f_exp


Control  SCZ  \
Feature                    Query                                        
near_gens_Annotation       coding nonsyn                      23   43   
                           coding syn                         16   20   
                           missense                            0    0   
                           stop-gain                           1    0   
                           intronic (splice_site)              0    1   
ensembl_Predicted Function coding                             39   64   
sift_Prediction            Deleterious                         8    9   
                           Deleterious - Low Confidence        1    1   
polyphen_Prediction        Probably Damaging                   3    5   
                           Possibly Damaging                   2    2   
tfbs_TFBS Name             any                                48   78   
phast_Score                any                               102  174   
gerp_Element RS Score      any                               140  262   
cpg_CpG Island             any                                26   47   
near_gens_Overlapped Gene  ADHD reported                       2    3   
                           ADHD                                2    3   
                           Anorexia nervosa reported           1    1   
                           Anorexia nervosa                    1    1   
                           ASD reported                        0    6   
                           ASD                                 2   20   
                           Bipolar disorder reported          23   53   
                           Bipolar disorder                   28   63   
                           MDD reported                        0    0   
                           MDD                                13   27   
                           Schizophrenia reported             11   43   
                           Schizophrenia                      11   43   
                           Psychiatric reported               34   97   
                           Psychiatric                        49  127   

                                                         chisq stat   chisq p  
Feature                    Query                                               
near_gens_Annotation       coding nonsyn                   0.707206  0.400373  
                           coding syn                      0.307556  0.579184  
                           missense                             NaN       NaN  
                           stop-gain                       1.505142  0.219882  
                           intronic (splice_site)          0.664389  0.415015  
ensembl_Predicted Function coding                          0.181154  0.670383  
sift_Prediction            Deleterious                     0.361448  0.547703  
                           Deleterious - Low Confidence    0.084766  0.770940  
polyphen_Prediction        Probably Damaging               0.019501  0.888940  
                           Possibly Damaging               0.169531  0.680529  
tfbs_TFBS Name             any                             0.174529  0.676118  
phast_Score                any                             1.009212  0.315092  
gerp_Element RS Score      any                             4.346044  0.037095  
cpg_CpG Island             any                             0.563172  0.452985  
near_gens_Overlapped Gene  ADHD reported                   0.000014  0.997009  
                           ADHD                            0.000014  0.997009  
                           Anorexia nervosa reported       0.084766  0.770940  
                           Anorexia nervosa                0.084766  0.770940  
                           ASD reported                    3.986334  0.045871  
                           ASD                             8.717100  0.003152  
                           Bipolar disorder reported       2.953804  0.085676  
                          

In [14]:
%connect_info

{
  "shell_port": 57503,
  "iopub_port": 47977,
  "stdin_port": 44435,
  "control_port": 44215,
  "hb_port": 42639,
  "ip": "127.0.0.1",
  "key": "d075aebb-e332663a3a5aa2278f13a72d",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-024de690-ae5c-46d6-b676-be47cebc5665.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
